In [1]:
import onnx
import torch
import torch.onnx

In [2]:
class AddConst(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w = 5

    def forward(self, x):
        return x + self.w


In [3]:
model = AddConst()
x = torch.randn(3, 4)
torch.onnx.export(model, x, "add_const.onnx", verbose=True)

Exported graph: graph(%onnx::Add_0 : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %/Constant_output_0 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={5}, onnx_name="/Constant"](), scope: __main__.AddConst:: # /var/folders/vw/rrzy8ptj6k10mvc6tf4170gw0000gq/T/ipykernel_45008/2842780031.py:7:0
  %2 : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu) = onnx::Add[onnx_name="/Add"](%onnx::Add_0, %/Constant_output_0), scope: __main__.AddConst:: # /var/folders/vw/rrzy8ptj6k10mvc6tf4170gw0000gq/T/ipykernel_45008/2842780031.py:7:0
  return (%2)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [7]:
net = onnx.load("add_const.onnx")
len(net.graph.node)

2

In [30]:
import numpy as np

In [31]:
np.frombuffer(net.graph.node[0].attribute[0].t.raw_data, dtype=np.float32)

array([5.], dtype=float32)